In [ ]:
import pandas as pd
import numpy as np
from openpyxl import load_workbook
import xgboost as xgb
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d

filename = "datasets/original_dataset.xlsx"
df = pd.read_excel(filename, engine='openpyxl')

df

In [ ]:

# 列出数值型特征
numeric_features = ['Long_1', 'Long_2', 'Long_3', 'Long_4', 'Long_5', 'Long_6',
                    'Long_7', 'Long_8', 'Long_9', 'Long_10'
                    'Short_1', 'Short_2', 'Short_3', 'Short_4', 'Short_5',
                    'Short_6', 'Short_7', 'Short_8', 'Short_9', 'Short_10']

# 使用线性插值来填充缺失值
for feature in numeric_features:
    # 创建一个插值函数，这里使用线性插值
    interpolator = interp1d(df['submission_year'], df[feature], kind='linear', fill_value='extrapolate')
    
    # 使用插值函数填充缺失值，并将填充结果转换为Series
    interpolated_values = pd.Series(interpolator(df['submission_year']), name=feature)
    
    # 使用fillna方法填充缺失值
    df[feature].fillna(interpolated_values, inplace=True)

df

In [ ]:
# Impute the DataFrame
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy="median")
numeric_cols = df.select_dtypes(include=['number']).columns
df[numeric_cols] = imputer.fit_transform(df[numeric_cols])


df

In [ ]:
# Separate data into training and validation sets
train_df = df[df['TrainVal'] == 'Train_60']
val_df = df[df['TrainVal'] == 'Val_40']

# Separate features and target variable
X_train = train_df.drop(columns=['UniqueID', 'submission_year', 'target', 'TrainVal',
                                 'Short_1','Short_2','Short_3','Short_4','Short_5','Short_6',
                                 'Short_7','Short_8','Short_9', 'Short_10'])
y_train = train_df['target']

X_val = val_df.drop(columns=['UniqueID', 'submission_year', 'target', 'TrainVal',
                             'Short_1','Short_2','Short_3','Short_4','Short_5',
                             'Short_6','Short_7','Short_8','Short_9', 'Short_10'])
y_val = val_df['target']

In [ ]:
# Train the XGBoost model with hyper parameter tuning

# Define the parameter grid
param_grid = {
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [3, 4, 5, 6],
    'n_estimators': [50, 100, 150],
    'subsample': [0.8, 0.9, 1],
    'colsample_bytree': [0.8, 0.9, 1],
}
clf = xgb.XGBClassifier()
grid_search = GridSearchCV(clf, param_grid, scoring='roc_auc', cv=3, verbose=1)
grid_search.fit(X_train, y_train)

# Best hyperparameters
print(grid_search.best_params_)

best_clf = grid_search.best_estimator_

# Predict using the best model
# y_pred = best_clf.predict(X_val_new)



In [ ]:
# AUC
from sklearn.metrics import roc_curve, auc

# best_clf.fit(X_train_new, y_train)

# Predict on validation set
y_pred = best_clf.predict_proba(X_val)[:, 1]
fpr, tpr, thresholds = roc_curve(y_val, y_pred)
roc_auc = auc(fpr, tpr)
plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rategit ')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC)')
plt.legend(loc="lower right")
plt.savefig('./images/baseline_result.jpg')

plt.show()